In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [4]:
import torch

In [5]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
dataset=load_dataset("imdb")

In [ ]:
# dataset

In [ ]:
train_reviews=dataset["train"]["text"]
train_labels=dataset["train"]["label"]
test_reviews=dataset["test"]["text"]
test_labels=dataset["test"]["label"]

In [ ]:
train_reviews[0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
train_labels[0]

0

In [ ]:
import pandas as pd
df=pd.DataFrame(train_reviews,columns=["text"])
df["label"]=train_labels
df["label"].value_counts()

,count
label,
0,12500
1,12500


In [ ]:
# !pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model=SentenceTransformer('all-MiniLM-L6-v2')
train_embeddings=model.encode(train_reviews,show_progress_bar=True)
test_embeddings=model.encode(test_reviews,show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

## **TensorDataset is used to bundle th independant and the dependant variables as one dataset of tpe tensor. DataLoader is used for batch processing**

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# !nvidia-smi

In [ ]:
X_train=torch.tensor(train_embeddings,dtype=torch.float32)
Y_train=torch.tensor(train_labels,dtype=torch.long)
X_test=torch.tensor(test_embeddings,dtype=torch.float32)
Y_test=torch.tensor(test_labels,dtype=torch.long)

In [ ]:
train_dataset=TensorDataset(X_train,Y_train)
test_dataset=TensorDataset(X_test,Y_test)

train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

# **Neural Nework with 2 HIDDEN LAYERS EACH WITH RELU ACTIVATION FUNCTION**

In [ ]:
from torch import nn

class Sentiment(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1=nn.Linear(384,128)
    self.layer2=nn.Linear(128,64)
    self.layer3=nn.Linear(64,1)
    self.relu=nn.ReLU()
  def forward(self,x):
    return self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))

# **DEVICE AGNOSTIC CODE**

In [ ]:

device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)
model=Sentiment()
# model.state_dict()
model.to(device)

Sentiment(
  (layer1): Linear(in_features=384, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

# **LOSS FUNCTION**

In [ ]:
Loss=nn.BCEWithLogitsLoss()
Optimizer=torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=0.01)

In [ ]:
# for x,y in test_loader:
#   print(y.size())

# **TRAINING**

In [ ]:
epochs=100
for epoch in range(epochs):
  model.train()
  for x,y in train_loader:
    x, y = x.to(device), y.to(device)
    pred=model(x)
    # print(pred.shape)
    loss=Loss(pred.squeeze(),y.float())
    Optimizer.zero_grad()
    loss.backward()
    Optimizer.step()

  print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


Epoch: 1, Loss: 0.2243468314409256
Epoch: 2, Loss: 0.425222247838974
Epoch: 3, Loss: 0.3296670913696289
Epoch: 4, Loss: 0.6150881052017212
Epoch: 5, Loss: 0.286370187997818
Epoch: 6, Loss: 0.4342113137245178
Epoch: 7, Loss: 0.1328025460243225
Epoch: 8, Loss: 0.3671402633190155
Epoch: 9, Loss: 0.329371839761734
Epoch: 10, Loss: 0.22173239290714264
Epoch: 11, Loss: 0.3293033242225647
Epoch: 12, Loss: 0.47994711995124817
Epoch: 13, Loss: 0.5171692371368408
Epoch: 14, Loss: 0.5394586324691772
Epoch: 15, Loss: 0.21583104133605957
Epoch: 16, Loss: 0.3709835708141327
Epoch: 17, Loss: 0.4828442633152008
Epoch: 18, Loss: 0.4416094422340393
Epoch: 19, Loss: 0.43984395265579224
Epoch: 20, Loss: 0.5421642065048218
Epoch: 21, Loss: 0.31203505396842957
Epoch: 22, Loss: 0.5374070405960083
Epoch: 23, Loss: 0.3943846821784973
Epoch: 24, Loss: 0.38803642988204956
Epoch: 25, Loss: 0.3645058870315552
Epoch: 26, Loss: 0.3528118133544922
Epoch: 27, Loss: 0.40231093764305115
Epoch: 28, Loss: 0.39337208867073

# **EVALUATION**

In [ ]:
  model.eval()
  total=0
  correct=0
  with torch.inference_mode():
    for x,y in test_loader:
      x, y = x.to(device), y.to(device)
      pred=model(x)
      # print(pred.shape)

      loss=Loss(pred.squeeze(),y.float())
      prob=torch.round(torch.sigmoid(pred))
      # predi=torch.where(prob>0.5,1,0)
      # print(pred)
      total+=y.size(0)
      correct += (prob.squeeze() == y).sum().item()

accuracy=correct/total

print(f"Accuracy: {accuracy}")
prob==y

Accuracy: 0.80244


tensor([[ True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True]],
       device='cuda:0')

# **Saving Model Parameters**

In [3]:
from pathlib import Path
MODEL_PATH=Path("models")
MODEL_PATH.mkdir(parents=True,exist_ok=True)
MODEL_NAME="Sentiment_Analysis.pth"
MODEL_SAVE_LOCATION=MODEL_PATH/MODEL_NAME
MODEL_SAVE_LOCATION

torch.save(obj=model.state_dict(),f=MODEL_SAVE_LOCATION)

NameError: name 'torch' is not defined

In [ ]:
user=input("Enter the review: ")

Enter the review: this is a beutiful day


In [ ]:
model=SentenceTransformer('all-MiniLM-L6-v2')
trains_embeddings=model.encode(user,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
user=torch.tensor(trains_embeddings,dtype=torch.float32)
Y_train=torch.tensor(train_labels,dtype=torch.long)

In [ ]:
model2=Sentiment()

In [ ]:
model2.load_state_dict(torch.load(f=MODEL_SAVE_LOCATION))

<All keys matched successfully>

In [ ]:
model2.eval()
with torch.inference_mode():
  pred=model2(user)
  pred_labels = torch.round(torch.sigmoid(pred))
  print(pred)
  print(pred_labels)
if pred_labels.item()==0.0:
  print("Negative Statement")
else:
  print("Positive Statement")

tensor([2.1564])
tensor([1.])
Positive Statement
